In [1]:
import subprocess  # For controlling volume on Mac
import pyautogui
import time
import math
import numpy as np
import mediapipe as mp
import cv2
from HandTrackingModule import handDetector  # Assuming you have the handTrackingModule as a separate file

# Setup for webcam and hand tracking
wCam, hCam = 640, 480
cap = cv2.VideoCapture(1)  # Ensure the correct camera index for Mac
cap.set(3, wCam)
cap.set(4, hCam)

# Initialize the handDetector (as used in the second code for hand landmarks)
mpHands = mp.solutions.hands
hands = mpHands.Hands(min_detection_confidence=0.85, min_tracking_confidence=0.8)
mpDraw = mp.solutions.drawing_utils

# Volume control function for Mac using subprocess
def set_volume(volume_level):
    # Ensure the volume is between 0 and 100
    volume_level = max(0, min(100, volume_level))
    print(f"Setting volume to {volume_level}")  # Debugging line
    subprocess.run(["osascript", "-e", f"set volume output volume {volume_level}"])

# Setup for controlling the screen size
tipIds = [4, 8, 12, 16, 20]
mode = ''
active = 0

pyautogui.FAILSAFE = False

# Function to put text on the image
def putText(mode, loc=(250, 450), color=(0, 255, 255)):
    cv2.putText(img, str(mode), loc, cv2.FONT_HERSHEY_COMPLEX_SMALL, 3, color, 3)

# Main loop
pTime = 0
while True:
    success, img = cap.read()
    if not success:
        print("Failed to grab frame")
        break

    # Convert image to RGB (since mediapipe expects RGB input)
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    # Draw hand landmarks if detected
    if results.multi_hand_landmarks:
        for landmarks in results.multi_hand_landmarks:
            mpDraw.draw_landmarks(img, landmarks, mpHands.HAND_CONNECTIONS)

    lmList = []
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for id, lm in enumerate(hand_landmarks.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmList.append([id, cx, cy])

    fingers = []

    if len(lmList) != 0:
        # Thumb logic
        if lmList[tipIds[0]][1] > lmList[tipIds[0 - 1]][1]:
            if lmList[tipIds[0]][1] >= lmList[tipIds[0] - 1][1]:
                fingers.append(1)
            else:
                fingers.append(0)
        elif lmList[tipIds[0]][1] < lmList[tipIds[0 - 1]][1]:
            if lmList[tipIds[0]][1] <= lmList[tipIds[0] - 1][1]:
                fingers.append(1)
            else:
                fingers.append(0)

        # Finger logic
        for id in range(1, 5):
            if lmList[tipIds[id]][2] < lmList[tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)

        # Mode detection
        if fingers == [0, 0, 0, 0, 0] and active == 0:
            mode = 'N'
        elif (fingers == [0, 1, 0, 0, 0] or fingers == [0, 1, 1, 0, 0]) and active == 0:
            mode = 'Scroll'
            active = 1
        elif (fingers == [1, 1, 0, 0, 0]) and active == 0:
            mode = 'Volume'
            active = 1
        elif (fingers == [1, 1, 1, 1, 1]) and active == 0:
            mode = 'Cursor'
            active = 1

    # Scroll Mode
    if mode == 'Scroll':
        active = 1
        putText(mode)
        cv2.rectangle(img, (200, 410), (245, 460), (255, 255, 255), cv2.FILLED)
        if len(lmList) != 0:
            if fingers == [0, 1, 0, 0, 0]:
                putText(mode='U', loc=(200, 455), color=(0, 255, 0))
                pyautogui.scroll(300)

            if fingers == [0, 1, 1, 0, 0]:
                putText(mode='D', loc=(200, 455), color=(0, 0, 255))
                pyautogui.scroll(-300)
            elif fingers == [0, 0, 0, 0, 0]:
                active = 0
                mode = 'N'

    # Volume Mode
    if mode == 'Volume':
        active = 1
        putText(mode)
        if len(lmList) != 0:
            if fingers[-1] == 1:
                active = 0
                mode = 'N'
                print(mode)
            else:
                x1, y1 = lmList[4][1], lmList[4][2]
                x2, y2 = lmList[8][1], lmList[8][2]
                cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
                cv2.circle(img, (x1, y1), 10, (0, 215, 255), cv2.FILLED)
                cv2.circle(img, (x2, y2), 10, (0, 215, 255), cv2.FILLED)
                cv2.line(img, (x1, y1), (x2, y2), (0, 215, 255), 3)
                cv2.circle(img, (cx, cy), 8, (0, 215, 255), cv2.FILLED)

                length = math.hypot(x2 - x1, y2 - y1)
                vol = np.interp(length, [50, 200], [0, 100])  # Volume scale from 0 to 100
                print(f"Hand length: {length}, Volume: {vol}")  # Debugging line
                set_volume(vol)

                if length < 50:
                    cv2.circle(img, (cx, cy), 11, (0, 0, 255), cv2.FILLED)

                cv2.rectangle(img, (30, 150), (55, 400), (209, 206, 0), 3)
                cv2.rectangle(img, (30, int(vol)), (55, 400), (215, 255, 127), cv2.FILLED)
                cv2.putText(img, f'{int(vol)}%', (25, 430), cv2.FONT_HERSHEY_COMPLEX, 0.9, (209, 206, 0), 3)

    # Cursor Mode
    if mode == 'Cursor':
        active = 1
        putText(mode)
        cv2.rectangle(img, (110, 20), (620, 350), (255, 255, 255), 3)

        if fingers[1:] == [0, 0, 0, 0]:  # Thumb excluded
            active = 0
            mode = 'N'
            print(mode)
        else:
            if len(lmList) != 0:
                x1, y1 = lmList[8][1], lmList[8][2]
                w, h = pyautogui.size()
                X = int(np.interp(x1, [110, 620], [0, w - 1]))
                Y = int(np.interp(y1, [20, 350], [0, h - 1]))
                cv2.circle(img, (lmList[8][1], lmList[8][2]), 7, (255, 255, 255), cv2.FILLED)
                cv2.circle(img, (lmList[4][1], lmList[4][2]), 10, (0, 255, 0), cv2.FILLED)  # Thumb

                if X % 2 != 0:
                    X = X - X % 2
                if Y % 2 != 0:
                    Y = Y - Y % 2
                pyautogui.moveTo(X, Y)

                if fingers[0] == 0:
                    cv2.circle(img, (lmList[4][1], lmList[4][2]), 10, (0, 0, 255), cv2.FILLED)
                    pyautogui.click()

    # Calculate FPS
    cTime = time.time()
    fps = 1 / ((cTime + 0.01) - pTime)
    pTime = cTime

    cv2.putText(img, f'FPS:{int(fps)}', (480, 50), cv2.FONT_ITALIC, 1, (255, 0, 0), 2)
    cv2.imshow('Hand LiveFeed', img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1740928559.475823  173982 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1740928559.505718  174332 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740928559.521844  174332 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740928566.192933  174332 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


N
Hand length: 38.600518131237564, Volume: 0.0
Setting volume to 0
Hand length: 39.45883931389772, Volume: 0.0
Setting volume to 0
Hand length: 61.5223536610881, Volume: 7.681569107392065
Setting volume to 7.681569107392065
Hand length: 81.2219182240853, Volume: 20.8146121493902
Setting volume to 20.8146121493902
Hand length: 91.0494371207203, Volume: 27.36629141381353
Setting volume to 27.36629141381353
Hand length: 97.082439194738, Volume: 31.388292796492
Setting volume to 31.388292796492
Hand length: 106.07544484940895, Volume: 37.383629899605964
Setting volume to 37.383629899605964
Hand length: 110.16351483136329, Volume: 40.109009887575525
Setting volume to 40.109009887575525
Hand length: 116.1077086157504, Volume: 44.07180574383359
Setting volume to 44.07180574383359
Hand length: 124.10076550932311, Volume: 49.40051033954874
Setting volume to 49.40051033954874
Hand length: 132.1854757528224, Volume: 54.79031716854827
Setting volume to 54.79031716854827
Hand length: 140.3566884761

Hand length: 47.29693436154187, Volume: 0.0
Setting volume to 0
Hand length: 43.41658669218482, Volume: 0.0
Setting volume to 0
Hand length: 55.65968020030299, Volume: 3.773120133535324
Setting volume to 3.773120133535324
Hand length: 73.49829930005184, Volume: 15.665532866701227
Setting volume to 15.665532866701227
Hand length: 131.21737689803132, Volume: 54.144917932020874
Setting volume to 54.144917932020874
N
N
N
N
N
N
N
